In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
import re

In [ ]:
import nltk                                      
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
df = pd.read_csv('/content/drive/My Drive/SI650/model_final.csv')

In [ ]:
lowered_summary = [k.lower() for k in df.summary]
df['summary'] = lowered_summary

In [ ]:
replaced = []
for i in df.summary:
  i = re.sub(r"[^ a-zA-Z0-9]+", "", i)
  replaced.append(i)

In [ ]:
df['summary'] = replaced

In [ ]:
stop = set(stopwords.words('english'))

In [ ]:
splitted = [k.split() for k in df.summary]

In [ ]:
new_words = []
for k in splitted:
  instance = []
  for l in k:
    if l not in stop:
      instance.append(l)
  new_words.append(instance)

In [ ]:
joined = []
for i in new_words:
  sentences = " ".join(i)
  joined.append(sentences)

In [ ]:
df['summary'] = joined

In [ ]:
df

,Unnamed: 0,ID,Content,summary,Title,Organization,Date,URL
0,0,0,Health Promotion Glossary World Health Organi...,health promotion glossary world health organiz...,Health Promotion Glossary,IFRC,1998-01-06T11:10:08Z,https://www.who.int/healthpromotion/about/HPR%...
1,1,1,Microsoft Word - health-policy.doc Health...,activities support health always central carry...,Microsoft Word - health-policy.doc,IFRC,2006-10-20T11:53:20Z,https://www.ifrc.org/Global/Publications/Healt...
2,2,2,The Use of Structured Observations in the Stu...,schools wideranging research programmes examin...,The Use of Structured Observations in the Stud...,IFRC,2000-12-06T13:57:01Z,https://www.ircwash.org/sites/default/files/Be...
3,4,4,Microsoft Word - JP7CIMCIProcessGuideFormat.d...,xiii written xiv guide organizedxv 1 purpose g...,Improving Health through Behavior Change A Pro...,IFRC,2004-08-27T18:49:28Z,http://www.ehproject.org/PDF/Joint_Publication...
4,5,5,"www.ifrc.org Saving lives, changing minds. vo...",40 million blood donations red cross red cresc...,Voluntary blood donation,IFRC,2012-12-04T08:08:25Z,https://www.ifrc.org/PageFiles/53503/1226403-I...
...,...,...,...,...,...,...,...,...
212,234,234,Leaving Millions Behind World Disasters Repo...,act disasters health emergencies meet needs im...,World Disasters Report 2018,IFRC,2018-11-27T09:22:32Z,http://media.ifrc.org/ifrc/wp-content/uploads/...
213,235,235,"Climate change, water and food security - ODI...",odi background notes provide summary snapshot ...,"Climate change, water and food security - ODI",IFRC,2009-03-11T09:04:07Z,https://www.odi.org/sites/odi.org.uk/files/odi...
214,236,236,Food Security and Nutrition Indicator Basics ...,level hunger decreased 27 millions people stil...,Food Security and Nutrition Indicator Basics,IFRC,2017-11-29T13:20:33Z,https://ifrcwatsanmissionassistant.files.wordp...
215,237,237,"IFRS - Group, Report (new structure) KPMG S...",ifrs group report new structure kpmg sa geneva...,IFRC Annual Report 2017,IFRC,2018-04-18T07:02:36Z,https://media.ifrc.org/ifrc/wp-content/uploads...


### Model Building

In [ ]:
!git clone https://github.com/deepset-ai/haystack.git
%cd haystack
!pip3 install --editable .

Cloning into 'haystack'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 7021 (delta 35), reused 46 (delta 26), pack-reused 6949
Receiving objects: 100% (7021/7021), 7.04 MiB | 27.42 MiB/s, done.
Resolving deltas: 100% (4889/4889), done.
/content/haystack
Obtaining file:///content/haystack
     |████████████████████████████████| 215kB 15.2MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 327kB 13.6MB/s 
     |████████████████████████████████| 327kB 40.6MB/s 
     |████████████████████████████████| 92kB 13.5MB/s 
     |████████████████████████████████| 983kB 57.1MB/s 
     |████████████████████████████████| 5.5MB 50.0MB/s 
     |████████████████████████████████| 143kB 51.1MB/s 
     |████████████████████████████████| 225kB 57.8MB/

In [ ]:
from haystack import Finder
from haystack.preprocessor.cleaning import clean_wiki_text
from haystack.preprocessor.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

12/10/2020 20:31:56 - INFO - faiss -   Loading faiss with AVX2 support.
12/10/2020 20:31:56 - INFO - faiss -   Loading faiss.


In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="shivika@icatalysts.net", password="MasterChariji1*", index="document")

12/10/2020 20:35:21 - INFO - elasticsearch -   HEAD http://localhost:9200/document [status:200 request:0.103s]
12/10/2020 20:35:21 - INFO - elasticsearch -   GET http://localhost:9200/document [status:200 request:0.011s]
12/10/2020 20:35:21 - INFO - elasticsearch -   PUT http://localhost:9200/document/_mapping [status:200 request:0.068s]
12/10/2020 20:35:21 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.008s]


In [ ]:
document_store.write_documents(df[['Title', 'summary', 'URL']].rename(columns={'Title':'name','summary':'text', 'URL': 'URL'}).to_dict(orient='records'))

12/10/2020 20:35:23 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.265s]


In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2-covid", use_gpu=True, context_window_size=1000)

12/10/2020 20:35:23 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/10/2020 20:35:23 - INFO - farm.infer -   Could not find `deepset/roberta-base-squad2-covid` locally. Try to download from model hub ...
12/10/2020 20:35:27 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
12/10/2020 20:35:31 - INFO - farm.utils -   device: cpu n_gpu: 0, distributed training: False, automatic mixed precision training: None
12/10/2020 20:35:31 - INFO - farm.infer -   Got ya 1 parallel workers to do inference ...
12/10/2020 20:35:31 - INFO - farm.infer -    0 
12/10/2020 20:35:31 - INFO - farm.infer -   /w\
12/10/2020 20:35:31 - INFO - farm.infer -   /'\
12/10/2020 20:35:31 - INFO - farm.infer -   


In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

In [ ]:
# x = retriever.eval(top_k_retrieval=10)

TypeError: ignored

In [ ]:
finder = Finder(reader, retriever)

12/10/2020 20:35:31 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.


In [ ]:
question = "water sanitation"
number_of_answers_to_fetch = 2

prediction = finder.get_answers(question=question, top_k_retriever=10, top_k_reader=number_of_answers_to_fetch)
print(f"Question: {prediction['question']}")
print("\n")
for i in range(number_of_answers_to_fetch):
    print(f"#{i+1}")
    print(f"Answer: {prediction['answers'][i]['answer']}")
    print(f"Research Paper: {prediction['answers'][i]['meta']['name']}")
    print(f"Context: {prediction['answers'][i]['context']}")
    print('\n\n')

12/10/2020 20:35:31 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
12/10/2020 20:35:31 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.124s]
12/10/2020 20:35:32 - INFO - haystack.finder -   Got 10 candidates from retriever
12/10/2020 20:35:32 - INFO - haystack.finder -   Reader is looking for detailed answer in 57902 chars ...
Inferencing Samples: 100%|██████████| 1/1 [00:04<00:00,  4.55s/ Batches]

Question: water sanitation


#1
Answer: however
Research Paper: Uganda Gender and Sanitation and Hygiene _GWA
Context: n views siting type structure access collected realize objective structures described put place mobilization implementation national guidelines sanitation spell roles groups requires follow sensitization ngos focussed approach gender mainstreaming projects end organization urges partners employ women staff specifically address sanitation hygiene issues women wilson p 2000 reports one partners wateraid gender unit others defined gender social objectives relation water sanitation work reported however genders gap representation decisionmaking power still persists especially communities deeply rooted cultural based gender biases conclusions enabling environment put place good starting point gender mainstreaming uganda gender sanitation hygiene gwadoc 17 mechanisms place promote gender longterm basis also routine whole organisation community level implementation structures

In [ ]:
question2 = "audit water safety plans"
number_of_answers_to_fetch2 = 2

prediction2 = finder.get_answers(question=question2, top_k_retriever=10, top_k_reader=number_of_answers_to_fetch2)
print(f"Question: {prediction2['question']}")
print("\n")
for i in range(number_of_answers_to_fetch2):
    print(f"#{i+1}")
    print(f"Answer: {prediction2['answers'][i]['answer']}")
    print(f"Research Paper: {prediction2['answers'][i]['meta']['name']}")
    print(f"Context: {prediction2['answers'][i]['context']}")
    print('\n\n')

12/08/2020 01:46:22 - WARNING - haystack.finder -   DEPRECATION WARNINGS: 
            1. The 'Finder' class will be deprecated in the next Haystack release in 
            favour of a new `Pipeline` class that supports building custom search pipelines using Haystack components
            including Retriever, Readers, and Generators.
            For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/544
            2. The `question` parameter in search requests & results is renamed to `query`.
12/08/2020 01:46:22 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.039s]
12/08/2020 01:46:22 - INFO - haystack.finder -   Got 10 candidates from retriever
12/08/2020 01:46:22 - INFO - haystack.finder -   Reader is looking for detailed answer in 946642 chars ...
Inferencing Samples: 100%|██████████| 4/4 [01:41<00:00, 25.35s/ Batches]

Question: audit water safety plans


#1
Answer: decreased global equity funds


KeyError: ignored

In [ ]:
prediction2

{'answers': [{'answer': 'decreased global equity funds',
   'context': 'nes equity investments held global equity trust fund listed equity trust fund invests actively traded equity securities mirror listed msci world index equity investments classified fair value profit loss 5 increase msci world index reporting date would increased global equity funds investment increased net surplus result increased total comprehensive income year chf 1250k 2015 chf 1141k reduction net deficit increase total comprehensive income equal change opposite direction would decreased global equity funds investment decreased net surplus result total comprehensive income year chf 1250k 2015 chf 1141k increase net deficit reduction total comprehensive income global bond fund investment classified fair value profit loss held listed fund indexed citigroup world government bonds index 5 increase index reporting date would increased global bond fund investment increased net surplus result increased total comprehens

In [ ]:
prediction2

{'answers': [{'answer': 'plans44 audit prearranged',
   'context': 'an address noncompliances complete noncompliant certificate water business details audit also reviews events occurred since last audit checks well wsp dealt recommendations areas auditor identified improvement could made amount nonconformance wsp audited water supply owned local authority run contractor water supply never achieved protozoa compliance due lack appropriate treatment occasionally noncompliant escherichia coli usually heavy rainfall practical guide auditing water safety plans44 audit prearranged six weeks advance audit participants cph auditor trainee auditor two representatives local authority water supplier two contracted operators supply approach followed allow sufficient time audit preparation whole audit process phase 1 report publication takes five six months legal requirement wsp approach takes form duty comprehensively risk assess water supply keep risk assessment review 2008 wsp approach became re

# Queries

In [ ]:
queries = pd.read_csv('/content/drive/My Drive/SI650/queries.csv')

In [ ]:
lowered_q = [k.lower() for k in queries.Query]
queries['Query'] = lowered_q

In [ ]:
queries

,Query_id,Query
0,1,how to audit water safety plans
1,2,annual summary reports
2,3,unicef reports
3,4,iwa reports
4,5,ifrc reports
5,6,funding for emergency
6,7,school wash
7,8,childhood development by education
8,9,audit of the consolidated financial statements
9,10,child protection policies
